# Visualizing time series data

This notebook provides the following methods for data visualization:
- Plotting time series
- Projecting time series as 2D (based on mean, max, sum)
- Reslice data based on the X_axis
- Clipping pixel values based on min & max quantile_thresholds
- Apply binary threshold mask
- Cropping data in x and y

In [1]:
from ipywidgets import IntSlider, interact, fixed, Dropdown, FloatRangeSlider, FloatText, IntRangeSlider
import numpy as np
import matplotlib.pyplot as plt
import tifffile 
from pathlib import Path

### 0_Define data path

In [2]:
datapath = list(Path('../../data/raw').iterdir())


### 1_Loading and plotting time series

In [3]:
def load_data(path):
    data = tifffile.TiffSequence(path).asarray()

    @interact(
        idx = IntSlider(value=0,min=0,max=data.shape[0]-1,continuous_frames =False)
    )
    def show(idx):
        plt.imshow(data[idx,:,:])

In [4]:
load_data(datapath)

interactive(children=(IntSlider(value=0, description='idx', max=2249), Output()), _dom_classes=('widget-intera…

### 2_Projecting time series as 2D
based on:
- max 
- mean
- sum

In [5]:

def summarize(path):
    data = tifffile.TiffSequence(path).asarray()
    @interact(
        method = ['mean', 'max', 'sum']
    )
    def show(method):
        if method == 'mean':
            plt.imshow(np.mean(data,axis =0))
        elif method == 'sum':
            plt.imshow(data.sum(axis=0))
        elif method == 'max':
            plt.imshow(np.max(data, axis=0))


In [6]:
summarize(datapath)

interactive(children=(Dropdown(description='method', options=('mean', 'max', 'sum'), value='mean'), Output()),…

### 3_reslice data based on the X_axis

In [7]:
def re_slice(path):
    data = tifffile.TiffSequence(path).asarray()

    @interact(
        idx = IntSlider(value=0,min=0,max=data.shape[1]-1,continuous_frames =False)
    )
    def show(idx):
        plt.imshow(data[:,idx,:])

In [8]:
re_slice(datapath)

interactive(children=(IntSlider(value=0, description='idx', max=497), Output()), _dom_classes=('widget-interac…

### NOTE: from here on we loaded the data first and then ran the other functions

In [9]:
def load_data_only(path):
    data = tifffile.TiffSequence(path).asarray()
    return data
test_data  = load_data_only(datapath)

### 4_Clipping pixel values based on min max thresholds

In [10]:
@interact(
    value = FloatRangeSlider(
                value=[0.1, 0.9],
                min=0,
                max=1,
                step=0.1,
                orientation='horizontal',
                readout=True,
                readout_format='.1f',
))
def data_remove_background(value):
    lower_limit = np.quantile(test_data, value[0])
    upper_limit = np.quantile(test_data, value[1])
    new_data = test_data.copy()
    new_data[new_data < lower_limit] = 0
    new_data[new_data > upper_limit] = upper_limit
    @interact(
    idx = IntSlider(value=0,min=0,max=new_data.shape[0]-1,continuous_frames =False)
    )
    def show(idx):
        plt.imshow(new_data[idx,:,:])
    

interactive(children=(FloatRangeSlider(value=(0.1, 0.9), description='value', max=1.0, readout_format='.1f'), …

### 5_Apply binary threshold mask

In [11]:
@interact(
    value = FloatText(
                value=0.5,
                min=0,
                max=1,
                step=0.1,
                orientation='horizontal',
                readout=True,
                readout_format='.1f',
))
def data_threshold(value):
    limit = np.quantile(test_data, value)
    new_data = np.where(test_data > limit, 1, 0)
    @interact(
    idx = IntSlider(value=0,min=0,max=new_data.shape[0]-1,continuous_frames =False)
    )
    def show(idx):
        plt.imshow(new_data[idx,:,:], cmap='gray')

interactive(children=(FloatText(value=0.5, description='value', step=0.1), Output()), _dom_classes=('widget-in…

### 6_Cropping data in x and y

In [12]:
@interact(
    x_slice = IntRangeSlider(
                value=[150, 350],
                min=0,
                max=test_data.shape[1] - 1,
                step=1,
                orientation='horizontal',
                readout=True,
                readout_format='.0f',
                continuous_update =False,
), 
    y_slice = IntRangeSlider(
                value=[150, 350],
                min=0,
                max=test_data.shape[2] - 1,
                step=1,
                orientation='horizontal',
                readout=True,
                readout_format='.0f',   
                continuous_update =False,
))
def slice_data(x_slice, y_slice):
    new_data = test_data[:, x_slice[0]:x_slice[1], y_slice[0]:y_slice[1]]
    
    @interact(
    idx = IntSlider(value=0,min=0,max=new_data.shape[0]-1,continuous_update =True),
    )
    def show(idx):
        plt.imshow(new_data[idx,:,:])
        plt.tight_layout()

interactive(children=(IntRangeSlider(value=(150, 350), continuous_update=False, description='x_slice', max=497…